In [23]:
import pandas as pd
import numpy as np
import collections
import operator

In [24]:
## Here goes the implementation for predicing haplogroups

In [25]:
df_intermediate = pd.read_csv("Hg_Prediction_tables/Intermediates.txt", header=None)
intermediates = df_intermediate[0].values

In [26]:
sample_name = "Output_example_datasets/Ancient_DNA_WGS/3DT26/3DT26.out"
df_haplogroup = pd.read_csv(sample_name, sep="\t")

In [27]:
df_haplogroup = df_haplogroup.sort_values(by=['haplogroup'])

In [28]:
#df_all = df_haplogroup

In [29]:
df_haplogroup['haplogroup'] = df_haplogroup['haplogroup'].str.replace('~', '')

### Derive State

#### Removes the intermediate haplogroups temporally

In [30]:
df_derived = df_haplogroup[df_haplogroup["state"] == "D"]

In [31]:
## instance with only D state
df_tmp = df_derived
for hg in intermediates:    
    df_tmp = df_tmp.drop(df_tmp[df_tmp.haplogroup == hg].index)
hg = df_tmp["haplogroup"].values

#### Choose the haplogroup based on the highest count in the first letter (E.x. J2a21 = J)

In [32]:
list_hg = []
for i in hg:
    list_hg.append(i)
collections_hg = collections.Counter(list_hg)

In [33]:
init_hg = max(collections_hg.items(), key=operator.itemgetter(1))[0]
init_hg = init_hg[0]

In [34]:
tmp_init_hg = init_hg+"_int.txt"
hg_intermediate_file = "Hg_Prediction_tables/"+tmp_init_hg
df_intermediate = pd.read_csv(hg_intermediate_file, header=None, sep="\t")

### QC.1 
Now you have the putative haplogroup. Open the file with the haplogroup intermediate and show the intermediate haplogroups again and compare if there are some where does not match with the correct state and store it a error. Give the estimate of the  correct_count/total  

$$QC1 = \frac{correct state } {total}$$

where 
###### correct state is the total number of correct haplogroup state found in the intermediate haplogroup file
###### total of all intermediate haplogroup found in the intermediate haplogroup file

In [35]:
total = 0
correct_state = 0
for i in df_intermediate.values:           
    tmp = df_haplogroup.loc[df_haplogroup["haplogroup"] == i[0]]    
    if not tmp.empty:               
        if "/" in i[1]:
            #i = i[1].split("/")[-1] 
            correct_state += len(tmp) 
            total += len(tmp)                           
        else:        
            correct_state += np.sum(i[1] == tmp["state"])                                                
            total += len(tmp)        
try:
    qc_one = round((correct_state / total),3)
except ZeroDivisionError as error:
    qc_one = 0.0

In [36]:
qc_one

0.998

Removes all haplogroup but the main one
Check if the preffix of all main haplogroup with D state by allowing one haplogroup that does not match to the main haplogroup until you get back to the origin. 
If mismatch > 1 skip choose as main haplogroup the following in the largest length and continue. In case haplogroup contains ~ symbol at the end ignore it temporally for preffix comparison but if this is the main haplogroup store the special character at the end. 

In [37]:
## Removes intermediate branches
df_haplogroup = df_haplogroup[~df_haplogroup.haplogroup.isin(intermediates)]

In [38]:
hg = df_derived[(df_derived.haplogroup.str.startswith(init_hg))].haplogroup.values

### Ancestral and Derive State

### QC.2

Check if the same name of the main haplogroup appears as an Ancestral State and save the number of count and calculate QC2:

$$QC2 = \frac{Dhg-Ahg} {total}$$
where 
###### Dhg is the total number of derived main haplogroup 
###### Ahg is the total number of ancestral with same name of main haplogroup
###### total of all derived and ancestral haplogroup with the same name

In [39]:
list_main_hg = sorted(list(set(hg)), reverse=False)
haplogroup_threshold = 0.70
qc_two = 0.0
qc_tmp = 0.0
dict_hg = {}

for putative_hg in list_main_hg:
    total_qctwo = len(df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg])
    Ahg = np.sum("A" == df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg]["state"])
    try:
        qc_tmp = round((total_qctwo-Ahg)/total_qctwo,3)
    except ZeroDivisionError as error:
        qc_tmp = 0.0
    if qc_tmp >= haplogroup_threshold:
        qc_two = qc_tmp                
        dict_hg[putative_hg] = qc_two    
        #print("{} {} ({} - {} / {})".format(putative_hg,qc_tmp, total_qctwo,Ahg,total_qctwo))
        #print(putative_hg)
    #else:
    #    print("{} {} ({} - {} / {})".format(putative_hg,qc_tmp, total_qctwo,Ahg,total_qctwo))
       
list_putative_hg = list(dict_hg.keys())
list_putative_hg.sort(reverse=True)
#print(list_putative_hg)
## Check for same pattern in the derive state
for i in range(len(list_putative_hg)-1):    
    if list_putative_hg[i+1] not in list_putative_hg[i]:                
        dict_hg.pop(list_putative_hg[i],None)
        #print(list_putative_hg[i])

if len(list_putative_hg) > 0:
    list_putative_hg = list(dict_hg.keys())
    putative_hg = max(dict_hg.keys(), key=len)    
    qc_two = dict_hg[putative_hg]
else:
    putative_hg = "NA"
    qc_two = 0.0


In [40]:
print(putative_hg)
print(qc_two)

J2b1
1.0


### QC.3
Show both Ancestral and Derived states from the main haplogroup and check the preffix of each haplogroup that is before the main one until you reach the root. If there are some Ancestral states which follows the pattern/preffix from the main haplogroup keep the count of how many of these appear and this will give the total count. Substract the ones found from the corrected and this will give the QC.3 score. 

In [41]:
if putative_hg != "NA":    
    list_main_hg_all = []
    list_hg_all = df_haplogroup["haplogroup"].values
    for i in list_hg_all:    
        if i.startswith(init_hg):
            list_main_hg_all.append(i)
    list_main_hg_all = sorted(list(set(list_main_hg_all)), reverse=True)
    df_main_hg_all = df_haplogroup.loc[df_haplogroup["haplogroup"].isin(list_main_hg_all)]
    df_main_hg_all = df_main_hg_all[["haplogroup","state","marker_name"]]
    df_main_hg_all = df_main_hg_all.sort_values(by="haplogroup", ascending=False).values

    qc_three = 0.0
    a_match = 0
    total_match = 0
    for putative_hg in [putative_hg]:
        for i in df_main_hg_all:
            tmp_hg = i[0].replace("~","")
            if tmp_hg in putative_hg:
                total_match +=1        
                if i[1] == "A":
                    a_match += 1
        qc_three = round((total_match - a_match) / total_match,3)        
else:
    qc_three = 0.0

print("{}: {}".format(putative_hg, qc_three))

J2b1: 0.978


### Final output
1) show the haplogroup name 
    if main haplogroup is a preffix from a higher resolution Ancestral state haplogroup report this haplogroup and marker name (Ex. HG: J21a Marker: L123)
    
2) Not J21a-L123

Could be that that there are more than one contain as a preffix from an ancestral state haplogroup. Should report all of them only if there are different haplogroup name with the resolution
**Check this in more detail later on... 

In [42]:
df_putative_ancestral_hg = df_haplogroup[df_haplogroup.haplogroup.str.startswith(putative_hg)]
df_putative_ancestral_hg = df_putative_ancestral_hg[df_putative_ancestral_hg.state == "A"]
putative_ancestral_hg = []        
for i in df_putative_ancestral_hg.index:    
    if putative_ancestral_hg == []:        
        putative_ancestral_hg.append(df_putative_ancestral_hg.loc[i])                    
    else:
        if putative_ancestral_hg[-1][3] not in df_putative_ancestral_hg.loc[i][3]:
            putative_ancestral_hg.append(df_putative_ancestral_hg.loc[i])
if len(putative_ancestral_hg) > 0:
    putative_ancestral_hg = pd.DataFrame(putative_ancestral_hg)

In [43]:
out_name = sample_name.split("/")[-1]
out_name = out_name.split(".")[0]
marker_name = (df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg]["marker_name"].values)
if putative_hg == "NA":
    out_hg = "NA"
else:
    if len(marker_name) > 1:
        out_hg = putative_hg[0]+"-"+marker_name[0]+"/etc"
    elif len(marker_name) == 1:
        out_hg = putative_hg[0]+"-"+marker_name[0]
    
header = "Sample_name\tHg\tHg_marker\tQC-score\tQC-1\tQC-2\tQC-3"

if len(putative_ancestral_hg) > 0:
    out_hg += "*(x"
    for i in putative_ancestral_hg.index:        
        out_hg += putative_ancestral_hg.loc[i]["marker_name"]+","
    out_hg += ")"

output = "{}\t{}\t{}\t{}\t{}\t{}\t{}".format(out_name,putative_hg,out_hg,round((qc_one*qc_two*qc_three),3),qc_one,qc_two,qc_three)
print(header)
print(output)

Sample_name	Hg	Hg_marker	QC-score	QC-1	QC-2	QC-3
3DT26	J2b1	J-M205	0.976	0.998	1.0	0.978
